In [1]:
import yaml
import os
os.chdir('..')
print(os.getcwd())

z:\OMOP\omop_etl


In [2]:
%run -i run_etl.py

Loading person ...
load_table complete. Elapsed time 00:00:11
100 rows affected
Loading death ...
load_table complete. Elapsed time 00:00:00
5 rows affected
Preloading condition_occurrence (None) ...
preload complete. Elapsed time 00:00:07
Loading condition_occurrence ...
load_table complete. Elapsed time 00:00:08
14240 rows affected
Preloading procedure_occurrence (icd) ...
preload complete. Elapsed time 00:00:07
Preloading procedure_occurrence (cpt) ...
preload complete. Elapsed time 00:00:10
Loading procedure_occurrence ...
load_table complete. Elapsed time 00:00:18
1552 rows affected
Loading drug_exposure ...
load_table complete. Elapsed time 00:00:00
16692 rows affected
z:\OMOP\omop_etl


In [4]:
from omop_etl.postproc.load import Loader, PRELOAD
load = Loader('config.yml')
# load.load_person()
# load.preload('measurement','height')

In [7]:
# load.load_table('condition_occurrence')
# load.update_mappings('person')
load.preload('condition_occurrence', subset='all')

True

In [8]:
# from omop_etl.datastore import read_sql
# read_sql('omop_etl/postproc/sql/person_mapping.sql')

In [3]:
# truncate('preload','drug_exposure', load.engine)

In [3]:
f = open('config.yml')
load = yaml.safe_load(f)
# print(preload)
# list(preload['preload']['drug'].keys())

In [4]:
STAGE = {
    'person': 'PERSON',
    'death': 'DEATH',
    'condition_occurrence': 'CONDITION',
    'procedure_occurrence': {
        'cpt': 'PROCEDURE_CPT',
        'icd': 'PROCEDURE_ICD'
    },
    'drug_exposure': {
        'order': 'DRUG_ADMIN', 
        'admin': 'DRUG_ORDER'
    },
    'measurement': {
        'bp': 'MEASUREMENT_BP',
        'res_pip': 'MEASUREMENT_Res_PIP',
        'heart_rate': 'MEASUREMENT_HeartRate',
        'height': 'MEASUREMENT_Height'
    }
}

dp_list = []
for t in load['load'].keys():
    if load['load'][t]:
        for part in load['load'][t].keys(): 
            dp_list.append(STAGE[t][part])
    else: 
        dp_list.append(STAGE[t])
        
print(dp_list)

['PERSON', 'DEATH', 'CONDITION', 'PROCEDURE_ICD', 'PROCEDURE_CPT', 'DRUG_ADMIN', 'DRUG_ORDER', 'MEASUREMENT_BP', 'MEASUREMENT_HeartRate', 'MEASUREMENT_Height']


In [6]:
from omop_etl.datastore import DataStore, execute
omop = DataStore('config.yml')
# t = omop.list_tables(in_schema=['preload'])

In [8]:
omop.config_param['load']['condition_occurrence']

In [21]:
# for table in t.Table:
#     execute(f'drop table if exists preload.{table}', omop.engine)

In [2]:
%run -i omop_etl/pullrawdata/pull_raw_data_v2.py

Start time: 2021-01-14 09:51:51.962589



In [1]:
a = False
if a == False:
    print('a is False')

a is False
